In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../MNIST_data/", one_hot = True)

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [40]:
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

In [4]:
n_input = 28 * 28
n_classes = 10

In [5]:
dropout = 0.75

In [6]:
X = tf.placeholder(tf.float32, [None, n_input])

In [7]:
# Change the form of 4D input images to a tensor
_X = tf.reshape(X, shape = [-1, 28, 28, 1])

In [8]:
y = tf.placeholder(tf.float32, [None, n_classes])

In [10]:
# First convolutional layer
wc1 = tf.Variable(tf.random_normal([5, 5, 1, 32]))
bc1 = tf.Variable(tf.random_normal([32]))

In [11]:
def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w, strides = [1, 1, 1, 1], padding = 'SAME'), b))

In [14]:
conv1 = conv2d(_X, wc1, bc1)

In [16]:
def max_pool(img, k):
    return tf.nn.max_pool(img, 
                          ksize = [1, k, k, 1],
                          strides = [1, k, k, 1],
                          padding = 'SAME')

In [17]:
conv1 = max_pool(conv1, k = 2)

In [18]:
keep_prob = tf.placeholder(tf.float32)
conv1 = tf.nn.dropout(conv1, keep_prob)

In [19]:
# Second convolutional layer
wc2 = tf.Variable(tf.random_normal([5, 5, 32, 64]))
bc2 = tf.Variable(tf.random_normal([64]))

In [20]:
conv2 = conv2d(conv1, wc2, bc2)

In [21]:
conv2 = max_pool(conv2, k = 2)

In [22]:
conv2 = tf.nn.dropout(conv2, keep_prob)

In [23]:
# Densely connected layer
wd1 = tf.Variable(tf.random_normal([7 * 7 * 64, 1024]))
bd1 = tf.Variable(tf.random_normal([1024]))

In [24]:
# Reshape conv2 output to fit dense layer input
dense1 = tf.reshape(conv2, [-1, wd1.get_shape().as_list()[0]])

In [25]:
# Relu activation
dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, wd1), bd1))

In [31]:
# Apply dropout
dense1 = tf.nn.dropout(dense1, keep_prob)

In [35]:
# Readout layer
wout = tf.Variable(tf.random_normal([1024, n_classes]))
bout = tf.Variable(tf.random_normal([n_classes]))

In [36]:
pred = tf.add(tf.matmul(dense1, wout), bout)

In [41]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

In [42]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [45]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict = {X: batch_xs, 
                                         y: batch_ys,
                                         keep_prob: dropout})
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict = {X: batch_xs,
                                                  y: batch_ys,
                                                  keep_prob: 1})
            loss = sess.run(cost, feed_dict = {X: batch_xs, 
                                               y: batch_ys,
                                               keep_prob: 1})
            print('Iter: {} Minibatch Loss = {:.6f} Training Accuracy = {:.5f}'.format(step * batch_size + 1, loss, acc))
        step += 1
    print('Optimization finished')
    print('Testing Accuracy: {}'.format(sess.run(accuracy, feed_dict = {X: mnist.test.images[:256],
                                                                        y: mnist.test.labels[:256],
                                                                        keep_prob: 1})))

Iter: 1281 Minibatch Loss = 29155.519531 Training Accuracy = 0.20312
Iter: 2561 Minibatch Loss = 31672.998047 Training Accuracy = 0.18750
Iter: 3841 Minibatch Loss = 12248.218750 Training Accuracy = 0.54688
Iter: 5121 Minibatch Loss = 8256.911133 Training Accuracy = 0.59375
Iter: 6401 Minibatch Loss = 8774.748047 Training Accuracy = 0.57812
Iter: 7681 Minibatch Loss = 7340.238281 Training Accuracy = 0.64062
Iter: 8961 Minibatch Loss = 5818.900879 Training Accuracy = 0.72656
Iter: 10241 Minibatch Loss = 4673.752441 Training Accuracy = 0.76562
Iter: 11521 Minibatch Loss = 6970.384277 Training Accuracy = 0.64844
Iter: 12801 Minibatch Loss = 2357.000977 Training Accuracy = 0.82031
Iter: 14081 Minibatch Loss = 1528.253540 Training Accuracy = 0.85156
Iter: 15361 Minibatch Loss = 3748.270996 Training Accuracy = 0.82812
Iter: 16641 Minibatch Loss = 2650.037598 Training Accuracy = 0.81250
Iter: 17921 Minibatch Loss = 3199.350098 Training Accuracy = 0.76562
Iter: 19201 Minibatch Loss = 1283.9907